In [1]:
import sys as _sys
import os

current_path = os.path.abspath(os.getcwd())

split = current_path.split("sgi_igl")
if len(split)<2:
    print("Please rename the repository 'sgi_igl'")
    raise ValueError
path_to_python_scripts = os.path.join(split[0], "sgi_igl/python/")
path_to_notifications = os.path.join(split[0], "sgi_igl/notebooks/notifications/")
path_to_settings = os.path.join(split[0], "sgi_igl/python/experiments/")
path_to_cubic_splines = os.path.join(split[0], "sgi_igl/ext/torchcubicspline/")
path_to_output = os.path.join(split[0], "sgi_igl/output/")
path_to_output_snake = os.path.join(path_to_output, "snake_all_joints/")

if not os.path.exists(path_to_output_snake):
    os.makedirs(path_to_output_snake)

_sys.path.insert(0, path_to_python_scripts)
_sys.path.insert(0, path_to_settings)
_sys.path.insert(0, path_to_cubic_splines)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import time

from geometry_io import export_snakes_to_json
from objectives import compare_last_translation, grad_compare_last_translation
from objectives import energy_path, grad_energy_path
from scipy.optimize import minimize, Bounds
from snake_shapes import snake_angles_generation
from step_backward_edge import multiple_steps_backward_material_pos_
from step_forward_edge import multiple_steps_forward
from utils import print_quaternion, axis_angle_to_quaternion
from vis_utils_snake import plot_animated_snake

TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)

# Define initial gait and material parameters

## Parameters

In [3]:
n_points_snake, n_ts = 11, 80
n_angles = n_points_snake - 2
edges = np.array([[idv, idv+1] for idv in range(n_points_snake-1)])
n_edges = edges.shape[0]
rho, eps, snake_length, close_snake_gait = 1.0e-2, 4.0e-2, 1.0, True
broken_joint_ids, broken_joint_angles = [], torch.tensor([])
target_translation = np.array([1.0 * snake_length, 0.0, 0.0])
target_rotation_axis = torch.tensor([0.0, 0.0, 0.0])
target_quaternion = axis_angle_to_quaternion(target_rotation_axis).numpy()
gt = np.concatenate([target_quaternion, target_translation], axis=0)
init_perturb_magnitude = 1.0e-2

broken_joint_angles = broken_joint_angles.reshape(1, -1) * torch.ones(size=(n_ts-close_snake_gait, 1))
n_op_angles = n_angles - len(broken_joint_ids)

edges_torch = torch.tensor(edges)
def fun_anisotropy_dir(x):
    tangents = x[..., edges_torch[:, 1], :] - x[..., edges_torch[:, 0], :]
    tangents = tangents / torch.linalg.norm(tangents, dim=-1, keepdims=True)
    return tangents

## Initial gait

In [4]:
operational_angles = torch.zeros(n_ts-close_snake_gait, n_op_angles)
torch.manual_seed(0)
operational_angles += init_perturb_magnitude * torch.randn(n_ts-close_snake_gait, n_op_angles)

example_pos_ = torch.zeros(size=(n_ts, n_points_snake, 3))
pos_ = snake_angles_generation(
    operational_angles, snake_length, broken_joint_ids, broken_joint_angles,
    example_pos_, n_ts, close_gait=close_snake_gait
).numpy()

tangents_ = fun_anisotropy_dir(torch.tensor(pos_)).numpy()

edge_lengths = np.linalg.norm(pos_[:, 1:] - pos_[:, :-1], axis=-1)
masses = rho * edge_lengths
a_weights = np.ones(shape=(n_ts, n_edges))
b_weights = (eps - 1.0) * np.ones(shape=(n_ts, n_edges))

force_0 = np.zeros(shape=(3,))
torque_0 = np.zeros(shape=(3,))

# Compute trajectory and save

In [5]:
pos, tangents, g = multiple_steps_forward(
    pos_, tangents_, masses, a_weights, b_weights, edges, force_0, torque_0, g0=np.array([0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0])
)

save_path = os.path.join(path_to_output_snake, "hello_world.json")

export_snakes_to_json(
    pos_, g, pos, force_0, torque_0, save_path, edges=None,
    weights_optim=None, quantities_per_vertex=None,
    quantities_per_edge=None, target_final_g=gt,
)

In [6]:
alphas = np.linspace(0.1, 1.0, n_ts)
fig = plt.figure(figsize=(10, 6))
gs = fig.add_gridspec(1, 1)
ax_tmp = fig.add_subplot(gs[0, 0])
for id_step in range(n_ts):
    ax_tmp.plot(pos[id_step, :, 0], pos[id_step, :, 1], lw=3.0, c='tab:blue', alpha=alphas[id_step], zorder=0)
    ax_tmp.scatter(g[id_step, 4], g[id_step, 5], marker='x', s=30.0, c='tab:blue', alpha=alphas[id_step], zorder=0)
ax_tmp.scatter(gt[4], gt[5], marker='o', s=30.0, c='tab:orange', alpha=1.0, zorder=1)

ax_tmp.set_aspect('equal')
plt.axis('off')
plt.show()
